In [2]:
%pip install pyriemann-qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 599.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 3.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 8.0 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... do

In [4]:
%pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 1.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
from sklearn import svm

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from mne.decoding import UnsupervisedSpatialFilter

from src.utils import load_and_regularize_covariance_matrices, calculate_covariance_means, project_to_tangent_and_triu, evaluate_metrics_repeatedly, reduce_spd_dataset
from src.covariance_means import generalized_eigenvalue_covariance_mean
from pyriemann_qiskit.classification import QuanticSVM, QuanticVQC
from pyriemann.classification import MDM
from pyriemann.channelselection import ElectrodeSelection

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def apply_spatialfiltering(covs, new_n=10):
    n_subject, n_video, n, _ = np.array(covs).shape
    sf = ElectrodeSelection(nelec=new_n)
    c = np.array(covs).reshape((n_subject * n_video, n, n))
    new_c = sf.fit(c).transform(c).reshape((n_subject, n_video, new_n, new_n))
    return new_c


In [8]:
c_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='CG')
c_dataset_covs = calculate_covariance_means(c_dataset) #Generalized eigenvalue covariance mean
c_dataset_covs = reduce_spd_dataset(c_dataset_covs, 3)
c_dataset_vectors = project_to_tangent_and_triu(c_dataset_covs)

pk_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='PG')
pk_dataset_covs = calculate_covariance_means(pk_dataset)
pk_dataset_covs = reduce_spd_dataset(pk_dataset_covs, 3)
pk_dataset_vectors = project_to_tangent_and_triu(pk_dataset_covs)

Patients:  36%|███▋      | 4/11 [00:00<00:00, 31.05it/s]

Patients [Tangent space]: 100%|██████████| 11/11 [00:00<00:00, 51.62it/s]


In [9]:
# Concatenate control and Pk
dataset_vectors = np.concatenate([c_dataset_vectors, pk_dataset_vectors], axis=0) # (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
dataset_labels = np.concatenate([np.zeros(c_dataset_vectors.shape[0]), np.ones(pk_dataset_vectors.shape[0])])

print(f"Dataset vectors shape: {dataset_vectors.shape}")

Dataset vectors shape: (22, 8, 6)


In [10]:
# Concatenate control and Pk
# pca = PCA(n_components=10)
# reduced_10dim_pkvectors = pca.fit_transform(pk_dataset_vectors)
# reduced_10dim_cvectors = pca.fit_transform(c_dataset_vectors)


# dataset_vectors = np.concatenate([c_dataset_vectors, pk_dataset_vectors], axis=0) 
# # (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
# # pca = UnsupervisedSpatialFilter(PCA(8, whiten=True), average=False)
# print(dataset_vectors.shape)
# n_samples, num_videos, num_features = dataset_vectors.shape

# # Reshape the dataset to 2D: (n_samples * num_videos, num_features)
# dataset_vectors_reshaped = dataset_vectors.reshape(n_samples * num_videos, num_features)

# # Check the shape of the reshaped data
# print(dataset_vectors_reshaped.shape)  # Should be (176, 210)


# pca = PCA(n_components=20)
# reduced10dim = pca.fit_transform(dataset_vectors_reshaped)
# res = reduced10dim.reshape(n_samples, num_videos, 20)
# print(reduced10dim.shape)
# print(res.shape)
# # dataset_vectors = np.concatenate([reduced_10dim_cvectors, reduced_10dim_pkvectors], axis=0) # (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
# dataset_labels = np.concatenate([np.zeros(c_dataset_vectors.shape[0]), np.ones(pk_dataset_vectors.shape[0])])


In [11]:
# Evaluate metrics
# evaluate_metrics_repeatedly(dataset_vectors = dataset_vectors,
#                       dataset_labels = dataset_labels,                      
#                       estimator = svm.SVC())
evaluate_metrics_repeatedly(dataset_vectors = dataset_vectors,
                      dataset_labels = dataset_labels,                      
                      estimator = QuanticSVM())

[QClass]  Initializing Quantum Classifier


Repetitions:   0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:FidelityQuantumKernel skipped because of time.
                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 6)
[QClass]  Feature dimension =  6
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  456127
[QClass]  SVM initiating algorithm
[QClass]  Training...


                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Prediction finished.
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 6)
[QClass]  Feature dimension =  6
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  456127
[QClass]  SVM initiating algorithm
[QClass]  Training...


...

                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Prediction finished.
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 6)
[QClass]  Feature dimension =  6
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  456127
[QClass]  SVM initiating algorithm
[QClass]  Training...


Repetitions: 100%|██████████| 1/1 [01:17<00:00, 77.22s/it]

[QClass]  Prediction finished.


,Accuracy,std Accuracy,Precision,std Precision,Recall,std Recall,F1 Score,std F1 Score
0,0.505682,0.375903,0.5,0.301511,0.34375,0.342715,0.380529,0.332693
